In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('detail_content.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(5)

,구분,제목,질문,답변
0,노동 > 근로일반,회사분할로 퇴직금 일괄수령 후 재입사 형식 취한 경우 계속근로인지,저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서...,기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하...
1,노동 > 근로일반,학습지판매위탁 상담교사도 근로기준법상의 근로자인지,저는 학습지제작·판매회사인 甲회사와 위탁업무계약을 체결한 교육상담교사입니다. 저와 ...,「근로기준법」상 근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 ...
2,노동 > 근로일반,대학병원의 전공의가 근로기준법상의 근로자에 해당하는지,저는 甲대학병원에서 전공의 과정을 밟고 있는 레지던트입니다. 그런데 몇 달 전부터 ...,「근로기준법」제2조 제1항 1호에서는 ‘근로자’를 ‘직업의 종류와 관계없이 임금을 ...
3,노동 > 근로일반,학교법인이 운영하는 대학교의 시간강사가 근로자인지,대학교에서 시간강사로 위촉한 강사들이 근로자임을 전제로 근로복지공단에서 학교법인 앞...,「근로기준법」제2조 제1항 제1호는 “‘근로자’란 직업의 종류와 관계없이 임금을 목...
4,노동 > 근로일반,자기소유 버스로 사업주의 회원 운송을 하면 근로기준법상 근로자인지,저는 제 소유 버스를 수영장 사업주의 명의로 등록하고 위 수영장에 전속되어 위 수영...,「근로기준법」상의 근로자에 해당하는지 여부에 관하여 판례는 “근로기준법상의 근로자에...


In [2]:
df.shape

(10047, 4)

In [3]:
print(df.질문[0])

저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서 일방적으로 근로자들을 일괄 사직처리하고 퇴직금을 수령하도록 한 후, 그 다음 날짜로 乙회사에 입사한 것으로 하였습니다. 그 후 제가 乙회사를 퇴직하게 되자 乙회사에서는 乙회사 근무기간 4년 9개월에 해당되는 퇴직금만을 지급하겠다고 합니다. 이 경우 甲회사에 입사한 때부터 乙회사에서 퇴직할 때까지의 기간을 계속근로연수로 하여 지급배수를 정한 누진퇴직금을 받을 수는 없는지요?


In [4]:
df.isna().sum().sum()

0

In [5]:
df.shape, df.제목.nunique(), df.질문.nunique()

((10047, 4), 8907, 9029)

In [6]:
df.drop_duplicates(subset=['제목'], inplace=True)
df.drop_duplicates(subset=['질문'], inplace=True)
df.shape

(8903, 4)

In [7]:
df.to_csv('detail_content_fin.tsv', sep='\t', index=False)

In [8]:
with open('한글불용어100.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [14]:
from tqdm import tqdm
contents = []
for content in tqdm(df.질문):
    morphs = okt.morphs(content, stem=True)
    clean_morph_content = ' '.join([morph for morph in morphs if morph not in stop_words])
    contents.append(clean_morph_content)

100%|██████████| 8903/8903 [00:36<00:00, 246.16it/s]


In [16]:
df['content'] = contents
df.to_csv('detail_content_word.tsv', index=False)

In [22]:
from tqdm import tqdm
answers = []
for content in tqdm(df.답변):
    morphs = okt.morphs(content, stem=True)
    clean_morph_content = ' '.join([morph for morph in morphs if morph not in stop_words])
    answers.append(clean_morph_content)

100%|██████████| 8903/8903 [03:35<00:00, 41.25it/s]


In [23]:
df['answer'] = answers
df.to_csv('detail_content_word.tsv', index=False)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [30]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import numpy as np

# Reshape the data to meet the requirements
contents = np.array(contents).reshape(-1, 1)
answers = np.array(answers)

# Now, you can apply the RandomOverSampler
oversampler = RandomOverSampler(random_state=2023)
X_resampled, y_resampled = oversampler.fit_resample(contents, answers)

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=2023)

pipeline = Pipeline([
    ('cvect', CountVectorizer()), ('nb', MultinomialNB())
])
params = {'cvect__ngram_range':[(1,1),(1,2)], 'cvect__max_df':[0.95,0.98]}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

ValueError: n_splits=3 cannot be greater than the number of members in each class.